In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import json
TOLERANCE = 1e-10
import pickle
import statistics
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt, dates as mdates
from scipy.stats import skew, kurtosis, norm
import statsmodels.api as sm
from scipy import stats
import seaborn as sns

In [ ]:
drive.mount('/content/drive')

In [ ]:
target = 0.18
fff = pd.read_csv('/content/drive/MyDrive/_Data_/ffm.csv', index_col = "dateff", parse_dates = True)[1:]
vix = pd.read_csv('/content/drive/MyDrive/_Data_/^VIX.csv', index_col = "Date", parse_dates = True)["Adj Close"]
rf = fff["rf"]
signals = []
for value in vix.values:
  if value > 30:
    signals.append("High")
  elif 15 < value <= 30:
    signals.append("Normal")
  else:
    signals.append("Low")

In [ ]:
with open("/content/drive/MyDrive/_Data_/ports_beta.json", "rb") as data:
    port_rets_beta = pickle.load(data)[:-1]

with open("/content/drive/MyDrive/_Data_/dates.json", "rb") as data:
    plot_dates = pickle.load(data)[12:]

**Full Sample**

In [ ]:
excess_returns = pd.DataFrame(port_rets_beta, index = plot_dates[1:], columns = ["Strategy Returns"]) 
excess_returns["Risk Free"] = rf.values
excess_returns["Excess Returns"] = excess_returns["Strategy Returns"] - excess_returns["Risk Free"]
rets = excess_returns["Excess Returns"].values

std = statistics.pstdev(rets) * np.sqrt(12) * 100
mean = np.mean(rets) * 12 * 100
skewness = skew(rets)
kurt = kurtosis(rets)
max = np.max(rets) * 100
min = np.min(rets) * 100
std_negative = statistics.pstdev([ret for ret in rets if ret < 0])  * np.sqrt(12) * 100

print("Standard Deviation: " + str(std) + "%")
print("Average Annual Return: " + str(mean) + "%")
print("Sharpe Ratio: " + str(mean/std))
print("Sortino Ratio: " + str(mean/std_negative))
print("Largest Gain: " + str(max) + "%")
print("Largest Loss: " + str(min) + "%") 
print("Skewness: " + str(skewness))
print("Kurtosis: " + str(kurt))

In [ ]:
cum_ret = 1
cum_rets = [1]
for ret in rets:
  cum_ret = cum_ret * (1 + ret)
  cum_rets.append(cum_ret)

max_ret = 1
drawdowns = []
for ret in cum_rets:
  max_ret = np.max([max_ret, ret])
  drawdowns.append(-(max_ret - ret) / max_ret * 100)

df = pd.DataFrame(cum_rets, plot_dates, columns = ["Cumulative Return"])
df["Drawdown"] = drawdowns

print("Largest Drawdown: " + str(round(np.min(drawdowns), 2)) + "%")
print("Calmar Ratio: " + str(mean / -np.min(drawdowns)))

In [ ]:
total = len(rets) - 1
positive = len([ret for ret in rets if ret > 0.0])
print("%Positive Days: " + str(positive/total * 100))

In [ ]:
excess_returns["Volatility"] = signals[1:]
mean_low = np.mean(excess_returns[excess_returns["Volatility"] == "Low"]["Excess Returns"]) * 12
mean_medium = np.mean(excess_returns[excess_returns["Volatility"] == "Normal"]["Excess Returns"]) * 12
mean_high = np.mean(excess_returns[excess_returns["Volatility"] == "High"]["Excess Returns"]) * 12

std_low = excess_returns[excess_returns["Volatility"] == "Low"]["Excess Returns"].std() * np.sqrt(12)
std_medium = excess_returns[excess_returns["Volatility"] == "Normal"]["Excess Returns"].std() * np.sqrt(12)
std_high = excess_returns[excess_returns["Volatility"] == "High"]["Excess Returns"].std() * np.sqrt(12)

sharpe_low = mean_low / std_low
sharpe_medium = mean_medium / std_medium
sharpe_high = mean_high / std_high

print("Volatility     Return     Stdev    Sharpe Ratio")
print("    Low         " + str(round(mean_low, 3)) + "     " + str(round(std_low, 3)) + "       " + str(round(sharpe_low, 3)))
print("    Medium      " + str(round(mean_medium, 3)) + "     " + str(round(std_medium, 3)) + "       " + str(round(sharpe_medium, 3)))
print("    High        " + str(round(mean_high, 3)) + "     " + str(round(std_high, 3)) + "       " + str(round(sharpe_high, 3)))

**First Half**

In [ ]:
rets = excess_returns["Excess Returns"][:"2010"].values

std = statistics.pstdev(rets) * np.sqrt(12) * 100
mean = np.mean(rets) * 12 * 100
skewness = skew(rets)
kurt = kurtosis(rets)
max = np.max(rets) * 100
min = np.min(rets) * 100
std_negative = statistics.pstdev([ret for ret in rets if ret < 0])  * np.sqrt(12) * 100

print("Standard Deviation: " + str(std) + "%")
print("Average Annual Return: " + str(mean) + "%")
print("Sharpe Ratio: " + str(mean/std))
print("Sortino Ratio: " + str(mean/std_negative))
print("Largest Gain: " + str(max) + "%")
print("Largest Loss: " + str(min) + "%") 
print("Skewness: " + str(skewness))
print("Kurtosis: " + str(kurt))

In [ ]:
cum_ret = 1
cum_rets = [1]
for ret in rets:
  cum_ret = cum_ret * (1 + ret)
  cum_rets.append(cum_ret)

max_ret = 1
drawdowns = []
for ret in cum_rets:
  max_ret = np.max([max_ret, ret])
  drawdowns.append(-(max_ret - ret) / max_ret * 100)

df = pd.DataFrame(cum_rets, plot_dates[:144], columns = ["Cumulative Return"])
df["Drawdown"] = drawdowns

print("Largest Drawdown: " + str(round(np.min(drawdowns), 2)) + "%")
print("Calmar Ratio: " + str(mean / -np.min(drawdowns)))

In [ ]:
total = len(rets) - 1
positive = len([ret for ret in rets if ret > 0.0])
print("%Positive Days: " + str(positive/total * 100))

**Second Half**

In [ ]:
rets = excess_returns["Excess Returns"]["2010":].values

std = statistics.pstdev(rets) * np.sqrt(12) * 100
mean = np.mean(rets) * 12 * 100
skewness = skew(rets)
kurt = kurtosis(rets)
max = np.max(rets) * 100
min = np.min(rets) * 100
std_negative = statistics.pstdev([ret for ret in rets if ret < 0])  * np.sqrt(12) * 100

print("Standard Deviation: " + str(std) + "%")
print("Average Annual Return: " + str(mean) + "%")
print("Sharpe Ratio: " + str(mean/std))
print("Sortino Ratio: " + str(mean/std_negative))
print("Largest Gain: " + str(max) + "%")
print("Largest Loss: " + str(min) + "%") 
print("Skewness: " + str(skewness))
print("Kurtosis: " + str(kurt))

In [ ]:
cum_ret = 1
cum_rets = [1]
for ret in rets:
  cum_ret = cum_ret * (1 + ret)
  cum_rets.append(cum_ret)

max_ret = 1
drawdowns = []
for ret in cum_rets:
  max_ret = np.max([max_ret, ret])
  drawdowns.append(-(max_ret - ret) / max_ret * 100)

df = pd.DataFrame(cum_rets, plot_dates[143:], columns = ["Cumulative Return"])
df["Drawdown"] = drawdowns

print("Largest Drawdown: " + str(round(np.min(drawdowns), 2)) + "%")
print("Calmar Ratio: " + str(mean / -np.min(drawdowns)))

In [ ]:
total = len(rets) - 1
positive = len([ret for ret in rets if ret > 0.0])
print("%Positive Days: " + str(positive/total * 100))

**Dot-com bubble crash (1998-2003)**

In [ ]:
rets = excess_returns["Excess Returns"]["1998":"2004"].values

std = statistics.pstdev(rets) * np.sqrt(12) * 100
mean = np.mean(rets) * 12 * 100
skewness = skew(rets)
kurt = kurtosis(rets)
max = np.max(rets) * 100
min = np.min(rets) * 100
std_negative = statistics.pstdev([ret for ret in rets if ret < 0])  * np.sqrt(12) * 100

print("Standard Deviation: " + str(std) + "%")
print("Average Annual Return: " + str(mean) + "%")
print("Sharpe Ratio: " + str(mean/std))
print("Sortino Ratio: " + str(mean/std_negative))
print("Largest Gain: " + str(max) + "%")
print("Largest Loss: " + str(min) + "%") 
print("Skewness: " + str(skewness))
print("Kurtosis: " + str(kurt))

In [ ]:
cum_ret = 1
cum_rets = [1]
for ret in rets:
  cum_ret = cum_ret * (1 + ret)
  cum_rets.append(cum_ret)

max_ret = 1
drawdowns = []
for ret in cum_rets:
  max_ret = np.max([max_ret, ret])
  drawdowns.append(-(max_ret - ret) / max_ret * 100)

df = pd.DataFrame(cum_rets, columns = ["Cumulative Return"])
df["Drawdown"] = drawdowns

print("Largest Drawdown: " + str(round(np.min(drawdowns), 2)) + "%")
print("Calmar Ratio: " + str(mean / -np.min(drawdowns)))

In [ ]:
total = len(rets) - 1
positive = len([ret for ret in rets if ret > 0.0])
print("%Positive Days: " + str(positive/total * 100))

**2007-2008 Market Crash (2007-2010)**

In [ ]:
rets = excess_returns["Excess Returns"]["2007":"2011"].values

std = statistics.pstdev(rets) * np.sqrt(12) * 100
mean = np.mean(rets) * 12 * 100
skewness = skew(rets)
kurt = kurtosis(rets)
max = np.max(rets) * 100
min = np.min(rets) * 100
std_negative = statistics.pstdev([ret for ret in rets if ret < 0])  * np.sqrt(12) * 100

print("Standard Deviation: " + str(std) + "%")
print("Average Annual Return: " + str(mean) + "%")
print("Sharpe Ratio: " + str(mean/std))
print("Sortino Ratio: " + str(mean/std_negative))
print("Largest Gain: " + str(max) + "%")
print("Largest Loss: " + str(min) + "%") 
print("Skewness: " + str(skewness))
print("Kurtosis: " + str(kurt))

In [ ]:
cum_ret = 1
cum_rets = [1]
for ret in rets:
  cum_ret = cum_ret * (1 + ret)
  cum_rets.append(cum_ret)

max_ret = 1
drawdowns = []
for ret in cum_rets:
  max_ret = np.max([max_ret, ret])
  drawdowns.append(-(max_ret - ret) / max_ret * 100)

df = pd.DataFrame(cum_rets, columns = ["Cumulative Return"])
df["Drawdown"] = drawdowns

print("Largest Drawdown: " + str(round(np.min(drawdowns), 2)) + "%")
print("Calmar Ratio: " + str(mean / -np.min(drawdowns)))

In [ ]:
total = len(rets) - 1
positive = len([ret for ret in rets if ret > 0.0])
print("%Positive Days: " + str(positive/total * 100))

**Covid**

In [ ]:
rets = excess_returns["Excess Returns"]["2020":"2021"].values

std = statistics.pstdev(rets) * np.sqrt(12) * 100
mean = np.mean(rets) * 12 * 100
skewness = skew(rets)
kurt = kurtosis(rets)
max = np.max(rets) * 100
min = np.min(rets) * 100
std_negative = statistics.pstdev([ret for ret in rets if ret < 0])  * np.sqrt(12) * 100

print("Standard Deviation: " + str(std) + "%")
print("Average Annual Return: " + str(mean) + "%")
print("Sharpe Ratio: " + str(mean/std))
print("Sortino Ratio: " + str(mean/std_negative))
print("Largest Gain: " + str(max) + "%")
print("Largest Loss: " + str(min) + "%") 
print("Skewness: " + str(skewness))
print("Kurtosis: " + str(kurt))

In [ ]:
cum_ret = 1
cum_rets = [1]
for ret in rets:
  cum_ret = cum_ret * (1 + ret)
  cum_rets.append(cum_ret)

max_ret = 1
drawdowns = []
for ret in cum_rets:
  max_ret = np.max([max_ret, ret])
  drawdowns.append(-(max_ret - ret) / max_ret * 100)

df = pd.DataFrame(cum_rets, columns = ["Cumulative Return"])
df["Drawdown"] = drawdowns

print("Largest Drawdown: " + str(round(np.min(drawdowns), 2)) + "%")
print("Calmar Ratio: " + str(mean / -np.min(drawdowns)))

In [ ]:
dates = excess_returns["Excess Returns"]["2020":"2021"].index
plt.figure(figsize=(24,7))
plt.plot(dates, rets)

**Factor Regression**

In [ ]:
excess_returns = pd.DataFrame(port_rets_beta, index = plot_dates[1:], columns = ["Strategy Returns"]) 
excess_returns["Strategy Returns"] = excess_returns["Strategy Returns"] 
excess_returns["Risk Free"] = rf.values
excess_returns["Excess Returns"] = excess_returns["Strategy Returns"] - excess_returns["Risk Free"]
rets = excess_returns["Excess Returns"].values

mom = pd.read_csv('/content/drive/MyDrive/_Data_/momentum.csv', index_col = "dateff", parse_dates = True)[1:]

fff = fff
fff["Momentum"] = mom["umd"].values
fff["Excess Returns"] = rets

port_excess = fff["Excess Returns"]
mkt_excess = fff["mktrf"]
exp_var = pd.DataFrame()
exp_var["Market - Rf"] = mkt_excess.copy()
exp_var["Constant"] = 1
exp_var["Value"] = fff["hml"]
exp_var["Size"] = fff["smb"]
exp_var["Momentum"] = fff["Momentum"]
lm = sm.OLS(port_excess, exp_var).fit()

lm.summary()

In [ ]:
replicating_port = excess_returns["Strategy Returns"].values - lm.params[0]*(fff['mktrf'].values + fff['rf'].values) -(1-lm.params[0])*fff['rf'].values -lm.params[2]*fff['hml'].values-lm.params[3]*fff['smb'].values-lm.params[4]*fff['Momentum'].values

In [ ]:
te = replicating_port.std() 
alpha = np.mean(replicating_port) 
IR = alpha * 12 / (te * np.sqrt(12))

print("Information Ratio: " + str(IR))

**Transaction Costs**

In [ ]:
excess_returns = pd.DataFrame(port_rets_beta, index = plot_dates[1:], columns = ["Strategy Returns"]) 
excess_returns["Strategy Returns"] = excess_returns["Strategy Returns"] - 0.00461
excess_returns["Risk Free"] = rf.values
excess_returns["Excess Returns"] = excess_returns["Strategy Returns"] - excess_returns["Risk Free"]
rets = excess_returns["Excess Returns"].values

mom = pd.read_csv('/content/drive/MyDrive/_Data_/momentum.csv', index_col = "dateff", parse_dates = True)[1:]

fff = fff
fff["Momentum"] = mom["umd"].values
fff["Excess Returns"] = rets

port_excess = fff["Excess Returns"]
mkt_excess = fff["mktrf"]
exp_var = pd.DataFrame()
exp_var["Market - Rf"] = mkt_excess.copy()
exp_var["Constant"] = 1
exp_var["Value"] = fff["hml"]
exp_var["Size"] = fff["smb"]
exp_var["Momentum"] = fff["Momentum"]
lm = sm.OLS(port_excess, exp_var).fit()

lm.summary()

**Historical VaR**

In [ ]:
rets = sorted(excess_returns["Excess Returns"].values)
length = len(rets)
index = round(length / 100, 0) - 1

var = rets[int(index)]

print("Historical VaR: " + str(var * 100) + "%")

**Sharpe Ratio across time**

In [ ]:
years = range(1998, 2023)
srs = []

for i in range(len(years)-1):
  rets = excess_returns['Excess Returns'][str(years[i]) : str(years[i+1])]
  mean = np.mean(rets)
  std = statistics.pstdev(rets)
  sr = mean/std * np.sqrt(12)
  srs.append(sr)

  print(str(years[i]) + ": " + str(sr))


rets = excess_returns['Excess Returns']["2022" : ]
mean = np.mean(rets)
std = statistics.pstdev(rets)
sr = mean/std * np.sqrt(12)
srs.append(sr)

print("2022: " + str(sr))

In [ ]:
df = pd.DataFrame(srs, index = years)
plt.figure(figsize=(24,7))
plt.plot(df)
plt.axhline(y=0, color = "black", alpha = 0.2)